# Linear Programming: Sensitivity Analysis

### Import dependencies

In [1]:
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer
from utils.answer_report import AnswerReport
from utils.sensitivity_report import SensitivityReport

### Sample Scenario

PRODUCTION. Pacific Aerospace is one of four subcontractors producing computer-controlled electrical switching assemblies for the proposed NASA space station. Pacific has the capability to produce three types of systems in-house: the Pacific Aerospace Delta, Omega, and Theta. NASA needs hundreds of all three systems and will purchase whatever Pacific chooses to produce.

All assemblies contain the tiny modified X70686 computer chip that cost Pacific \$500 to manufacture. Seven such chips are available daily. Other materials needed for the manufacture of the assembly cost \$200 for the Delta, \$400 for the Omega, and \$300 for the Theta, but they are not considered in short enough supply to restrict production. 

Each assembly must pass through a production center; it is then subjected to rigorous testing and quality control checks. The following table gives the relevant data for each assembly.

|                         | Delta          | Omega          | Theta          | Daily Availability |
|-------------------------|----------------|----------------|----------------|--------------------|
| Contract price          | $1500          | $1800          | $1400          | -                  |
|    X70686 Chip          | $500           | $500           | $500           | 7                  |
|    Other material/labor | $200           | $400           | $600           |                    |
|       Net profit        | $800           | $900           | $600           | -                  |
| Production (hrs.)       | 2              | 1              | 1              | 8                  |
| Quality checks (hrs.)   | $1\frac{1}{3}$ | $2\frac{2}{3}$ | $1\frac{1}{3}$ | 8                  |


a. Formulate and solve for the optimal daily production schedule. Note that no Omega systems would be produced. Why not?

Let $x_1$ be the number of Delta systems to produce per day, \
$\quad\,\, x_2$ be the number of Omega systems to produce per day, and \
$\quad\,\, x_3$ be the number of Theta systems to produce per day

Then, from the given information, we can formulate the linear programming problem as follows.

\begin{align*}
\text{max} 
&\quad z = 800x_1 + 900x_2 + 600x_3 \\
\text{subject to} \\
& x_1 + x_2 + x_3 \leqslant 7 \\
& 2x_1 + x_2 + x_3 \leqslant 8 \\
& 4x_1 + 8x_2 + 4x_3 \leqslant 24 \\
& x_i \geqslant 0 \quad \forall i \in \{1, 2, 3\} \\
& x_i \in \mathbb{Z} \quad \forall i \in \{1, 2, 3\}
\end{align*}

In [2]:
## define LP problem

# create model
model = Model(name = 'Pacific Aerospace Production for NASA')

# define decision variables
x_1 = model.integer_var(name = 'x_1', lb = 0)
x_2 = model.integer_var(name = 'x_2', lb = 0)
x_3 = model.integer_var(name = 'x_3', lb = 0)

# set objective function
z = model.set_objective('max', 800*x_1 + 900*x_2 + 600*x_3)

# add constraints
c_1 = model.add_constraint(x_1 + x_2 + x_3 <= 7, ctname = 'c_1')
c_2 = model.add_constraint(2*x_1 + x_2 + x_3 <= 8, ctname = 'c_2')
c_3 = model.add_constraint(4*x_1 + 8*x_2 + 4*x_3 <= 24, ctname = 'c_3')

In [3]:
## solve LP problem

# get solution
solution = model.solve()

# show optimal solution and optimal value
solution.get_values(model.iter_variables()), solution.get_objective_value()

([2.0, 0, 4.0], 4000.0)

### Answer Report

In [4]:
# get answer report
answer_report = AnswerReport(model)

In [5]:
# show answer report for decision variables
answer_report.decision_variables()

,Final Value,Variable Type
Name,,
x_1,2.0,integer
x_2,0.0,integer
x_3,4.0,integer


In [6]:
# show answer report for constraints
answer_report.constraints()

,Status,Slack
Name,,
c_1,non-binding,1.0
c_2,binding,0.0
c_3,binding,0.0


### Sensitivity Report

In [7]:
## relax LP problem

# get LP relaxation problem
lp_relaxation = LinearRelaxer.make_relaxed_model(model)

# get solution of relaxed LP problem
solution = lp_relaxation.solve()

# show optimal solution and optimal value of relaxed LP problem
solution.get_values(lp_relaxation.iter_variables()), solution.get_objective_value()

([2.0, 0, 4.0], 4000.0)

In [8]:
# get sensitivity report of relaxed LP problem
sensitivity_report = SensitivityReport(lp_relaxation)

In [9]:
# show sensitivity report for decision variables
sensitivity_report.decision_variables()

,Final Value,Reduced Cost,Objective Coefficient,Allowable Increase,Allowable Decrease
Name,,,,,
x_1,2.0,0.0,800,100.0,200.0
x_2,0.0,-100.0,900,100.0,1e+20
x_3,4.0,0.0,600,200.0,33.33333333333337


In [10]:
# show sensitivity report for constraints
sensitivity_report.constraints()

,Final Value,Shadow Price,Constraint RHS,Allowable Increase,Allowable Decrease
Name,,,,,
c_1,6.0,-0.0,7,1e+20,1.0
c_2,8.0,200.0,8,4.0,2.0
c_3,24.0,100.0,24,4.0,8.0


After solving the problem, we have the optimal solution $(x^*_1, x^*_2, x^*_3) = (2, 0, 4)$ with the corresponding optimal value of $z^* = 4000$. 

Hence, the optimal daily production schedule for Pacific is to produce 2 Delta systems and 4 Theta systems, which would result in the optimal net profit of \$4000.

It is worth noting that no Omega systems would be produced. This is because the Omega assembly is not as profitable, compared to the Delta system and the Theta system. 

Indeed, if we consider a sensitivity report for the decision variables, we can see that the reduced cost of the Omega system is -100. This means if Pacific Aerospace were to include 1 unit of the Omega system in the production, then the daily net profit would be decreased by \$100.

Therefore, in order to maximize the profit, Pacific Aerospace should not produce any Omega system.

b. What is the minimum contract price that would initiate production of the Omega systems?

In order to answer this question, let's consider a sensitivity report for the decision variables.

In [11]:
# show sensitivity report for decision variables
sensitivity_report.decision_variables()

,Final Value,Reduced Cost,Objective Coefficient,Allowable Increase,Allowable Decrease
Name,,,,,
x_1,2.0,0.0,800,100.0,200.0
x_2,0.0,-100.0,900,100.0,1e+20
x_3,4.0,0.0,600,200.0,33.33333333333337


From the above sensitivity report, it shows that the reduced cost for Omega is -100, which, as mentioned in (a), means that the daily net profit would be decreased by \$100 if Pacific Aerospace decided to increase the production of Omega systems from 0 to 1 unit.

However, alternatively, the reduced cost of the Omega can also be interpreted as the amount by which the unit profit of the Omega would have to be reduced before the Omega could assume a positive value in the optimal production schedule. 

In other words, the Omega assembly currently does not attract enough profit to be included in the production. Consequently, in order for the Omega system to become a viable addition to production, its profit contribution needs to reduced by -\$100 or, to put it differently, improved by \$100.

Therefore, the minimum contract price that would initiate the production of the Omega systems is 900 - (-100) = \$1000.

c. What is the minimum X70686 availability for which the solution in (a) remains optimal?

In order to answer this question, let's consider a sensitivity report for the constraints.

In [12]:
# show sensitivity report for constraints
sensitivity_report.constraints()

,Final Value,Shadow Price,Constraint RHS,Allowable Increase,Allowable Decrease
Name,,,,,
c_1,6.0,-0.0,7,1e+20,1.0
c_2,8.0,200.0,8,4.0,2.0
c_3,24.0,100.0,24,4.0,8.0


From the above sensitivity report, it shows that the allowable decrease of the right-hand side of the first constraint (i.e., the X70686 availability constraint) is 1.

Therefore, the minimum X70686 availability for which the solution in (a) remains optimal is 7 - 1 = 6.

d. Suppose you have the option of improving the profit by instituting one of the following options. Which would be of most value to Pacific Aerospace?

i. Receiving, on a daily basis, six additional X70686 chips for \$3100 \
ii. Utilizing three extra production hours daily at a cost of \$525 (\$175 / hr.) \
iii. Utilizing one additional quality check hour daily at a cost of \$200 (\$200 / hr.) 

From the sensitivity report for constraints in (c), it shows that the shadow prices for the X70686 availability, production time, and quality check time are 0, 200, and 100, respectively.

This means that Pacific Aerospace would be unwilling to pay more than \$0, \$200, and \$100 for an additional unit of X70686 availability, production time, and quality check time, respectively.

In other words, option (i) and (iii) should not be picked as the cost per unit for extra X70686 chips and quality check time exceeds their respective shadow prices.

Therefore, option (i), utilizing three extra production hours daily at a cost of \$525, would be the most value to Pacific Aerospace.